In [6]:
# Import libraries
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sb
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

# Imported Fact Order dataset
factOrder_df = pd.read_csv('FactOrder.csv')


# Find number of rows and columns of the dataset
print(factOrder_df.shape)

(1399850, 16)


In [7]:
factOrder_df

,OrderNumber,OrderRowNumber,ArticleKey,ArticleName,Colour,ArticleGenre,Size,ArticleGroupName,Collection,SizeGroup,BabyJunior,ArticleMaterial,DateOrderedKey,QuantityOrdered,Amount,Cost
0,94444500,1,31182,Artikel 31182,NaN,Övrigt,NaN,VAT,Base,NaN,NaN,NaN,20180102,1.0,997.83,0.00
1,94543600,2,47942,Artikel 47942,red,DRESSES LS WOVEN,116/122,DRESSES,Main,6[116-122],3 KIDS,NaN,20180103,1.0,375.69,70.67
2,94543600,3,6763,Artikel 6763,light grey,SWEATSHIRTS,92/98,TOPS,Main,4[92-98],2 TODDLER,NaN,20180103,1.0,178.75,68.44
3,94543700,1,20870,Artikel 20870,offwhite,TEES LS,80/86,TOPS,Main,3[80-86],2 TODDLER,NaN,20180103,1.0,187.53,25.67
4,94543700,2,38070,Artikel 38070,light grey,SWEATSHIRTS,80/86,TOPS,Main,3[80-86],2 TODDLER,NaN,20180103,1.0,178.75,68.44
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1399845,116135100,3,56158,Artikel 56158,Blue,TEES LS,104/110,TOPS,Main,5[104-110],3 KIDS,JERSEY,20201223,1.0,99.04,47.75
1399846,116135100,4,33839,Artikel 33839,Blue,TROUSERS JERSEY,92/98,BOTTOMS,Main,4[92-98],2 TODDLER,JERSEY,20201223,1.0,99.04,46.61
1399847,116135100,5,54492,Artikel 54492,Offwhite,LEGGINGS,92/98,BOTTOMS,Pre,4[92-98],2 TODDLER,JERSEY,20201223,1.0,104.54,47.58
1399848,116135100,6,35487,Artikel 35487,Offwhite,TEES SS,104/110,TOPS,Pre,5[104-110],3 KIDS,JERSEY,20201223,1.0,99.04,45.55


In [8]:
# Get column datatypes and check for null values
factOrder_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1399850 entries, 0 to 1399849
Data columns (total 16 columns):
 #   Column            Non-Null Count    Dtype  
---  ------            --------------    -----  
 0   OrderNumber       1399850 non-null  int64  
 1   OrderRowNumber    1399850 non-null  int64  
 2   ArticleKey        1399850 non-null  int64  
 3   ArticleName       1399850 non-null  object 
 4   Colour            1385814 non-null  object 
 5   ArticleGenre      1399705 non-null  object 
 6   Size              1385789 non-null  object 
 7   ArticleGroupName  1399850 non-null  object 
 8   Collection        1399705 non-null  object 
 9   SizeGroup         1379516 non-null  object 
 10  BabyJunior        1379516 non-null  object 
 11  ArticleMaterial   1140321 non-null  object 
 12  DateOrderedKey    1399850 non-null  int64  
 13  QuantityOrdered   1399850 non-null  float64
 14  Amount            1399850 non-null  float64
 15  Cost              1399850 non-null  float64
dtype

In [9]:
# Check for null values
factOrder_df.isnull().sum()

OrderNumber              0
OrderRowNumber           0
ArticleKey               0
ArticleName              0
Colour               14036
ArticleGenre           145
Size                 14061
ArticleGroupName         0
Collection             145
SizeGroup            20334
BabyJunior           20334
ArticleMaterial     259529
DateOrderedKey           0
QuantityOrdered          0
Amount                   0
Cost                     0
dtype: int64

In [10]:
# Check for duplicates
factOrder_df_dup = factOrder_df[factOrder_df.duplicated(keep='first')]
factOrder_df_dup

,OrderNumber,OrderRowNumber,ArticleKey,ArticleName,Colour,ArticleGenre,Size,ArticleGroupName,Collection,SizeGroup,BabyJunior,ArticleMaterial,DateOrderedKey,QuantityOrdered,Amount,Cost
